In [58]:
import pandas as pd
from simplification.cutil import simplify_coords, simplify_coords_vw, simplify_coords_vwp
import numpy as np
import simplify

In [21]:
"""Finds indices of bounces in the points list instead of the simplified trajectory."""
def find_indices(points, sp, idx):
    ix = []
    for index in idx:
        ix.append(points.index(sp[index]))
    return ix

In [14]:
def angle(dir, points):

    """Finds angles between lines in the simplified trajectory."""

    dir2 = dir[1:]
    dir1 = dir[:-1]
    radians = np.arccos(
    (dir1*dir2).sum(axis=1)/(np.sqrt((dir1**2).sum(axis=1)*(dir2**2).sum(axis=1))))

    degrees = np.degrees(radians)
    
    # Vectors for quandrant check
    vectors = []
    for i in range(len(points) - 2):
        p1 = (points[i])
        p2 = (points[i + 1])
        p3 = (points[i + 2])

        vector1 = np.array([p1[0] - p2[0], p1[1] - p2[1]])
        vector2 = np.array([p3[0] - p2[0], p3[1] - p2[1]])

        vectors.append((vector1, vector2))

    # Quadrant check
    is_bounce = []

    """
    触地和击打
    """
    
    for i in range(0, len(vectors)):
        # Vectors
        first = np.array(vectors[i][0])
        second = np.array(vectors[i][1])
        # is_bounce.append(True)

        v1_x_sign = 1 if first[0] > 0 else -1
        v1_y_sign = 1 if first[1] > 0 else -1
        v2_x_sign = 1 if second[0] > 0 else -1
        v2_y_sign = 1 if second[1] > 0 else -1
            
        bounce = True
            
        # Valleys are parabola peaks because of inverted screen 
        if (v1_y_sign == v2_y_sign and v1_y_sign == 1 and v1_x_sign != v2_x_sign):
            delta_theta = np.degrees(np.arccos(np.dot(first, second)/(np.linalg.norm(first)*np.linalg.norm(second))))
            print(delta_theta)
            if np.abs(delta_theta) < 95 and np.abs(delta_theta) > 20 and (np.linalg.norm(first) > 70 or np.linalg.norm(second) > 70):
                is_bounce.append('shot')
                # print("shot")
            else:
                is_bounce.append(False)
                # print("False")
        elif (v1_y_sign == v2_y_sign and v1_y_sign == -1 and v1_x_sign != v2_x_sign):
            is_bounce.append("bounce") 
        else:
            ang1 = np.arctan(first[1]/first[0])
            ang2 = np.arctan(second[1]/second[0]) 
            if v1_x_sign == 1 and v1_y_sign == 1 and v2_x_sign == -1 and v2_y_sign == -1 : # First vector in first quadrant, second vector in third quadrant
                if ang2 < ang1:
                    is_bounce.append(False)
                else:
                    is_bounce.append("shot")        # 115
                    print(f"Num {i} frame: 1 -> 3")
            elif v1_x_sign == -1 and v1_y_sign == 1 and v2_x_sign == 1 and v2_y_sign == -1: # First vector in second quadrant, second vector in fourth quadrant
                if np.abs(ang2) < np.abs(ang1):
                    is_bounce.append(False)
                else:
                    is_bounce.append("shot")
                    print(f"Num {i} frame: 2 -> 4")
            elif v1_x_sign == -1 and v1_y_sign == -1 and v2_x_sign == 1 and v2_y_sign == 1: # First vector in third quadrant, second vector in first quadrant
                if ang2 > ang1:
                    is_bounce.append(False)
                else:
                    is_bounce.append("bounce")
                    print(f"Num {i} frame: 3 -> 1")
            elif v1_x_sign == 1 and v1_y_sign == -1 and v2_x_sign == -1 and v2_y_sign == 1: # First vector in fourth quadrant, second vector in second quadrant
                if np.abs(ang1) < np.abs(ang2):
                    is_bounce.append(False)
                else:
                    is_bounce.append("bounce")
                    print(f"Num {i} frame: 4 -> 2")
            else: # If both vectors on the same side then it is 99% a player hitting the ball
                is_bounce.append("shot")

        # print("Pair: ", first, second, "Bounce: ", bounce, ", Pair number: ", i)
            
    return degrees, is_bounce

In [6]:
csv_path = "E:/tennis_v2/inference/HRnet_v3_test_02.csv"
trajectory = pd.read_csv(csv_path)

In [10]:
x = trajectory.x
y = trajectory.y
points = list(zip(x, y))
points

[(627.6766090489118, 70.08357354524982),
 (601.1365091515598, 101.04802363442286),
 (573.2159452527188, 134.44740733439062),
 (545.2575209254805, 171.69462752680028),
 (518.1540953433919, 209.9087500812459),
 (489.6624347644244, 251.77426276640756),
 (462.3609870125188, 294.20018841215995),
 (434.59211064556416, 340.0034469183009),
 (405.9159820494153, 388.7436671320704),
 (382.9489727995152, 406.9351598086661),
 (358.646887566902, 410.7679093918505),
 (336.2543489976252, 415.7001329494101),
 (310.8033642812183, 423.4828467994117),
 (285.9169512563033, 433.67893958935633),
 (261.7125101451801, 445.1650681884669),
 (238.0962811743035, 457.7859487288755),
 (210.5485805342158, 475.9001334149235),
 (192.6409885044873, 484.8687715448023),
 (220.78117046224423, 438.5295798981181),
 (246.79962291089623, 398.2947051725556),
 (271.2649366018386, 360.6794503393442),
 (285.00446128196654, 336.1546183157999),
 (312.84291844552274, 297.9914377751721),
 (334.1668348772772, 267.68757296160715),
 (353

In [60]:
simplified_vw = np.array(simplify.simplify(points, 1))
simplified_vw

AttributeError: module 'simplify' has no attribute 'simplify'

In [55]:
min_angle = 25
# compute the direction vectors on the simplified curve
directions = np.diff(simplified_vw, axis=0)

# Check if the angle is predominantly in the first or fourth quadrant, and if each angle corresponds to a bounce
theta, is_bounce = angle(directions, simplified_vw)
sx, sy = simplified_vw.T

Num 7 frame: 4 -> 2
Num 8 frame: 4 -> 2
Num 15 frame: 4 -> 2
Num 19 frame: 2 -> 4
Num 21 frame: 2 -> 4
Num 22 frame: 2 -> 4
171.15601683121713
Num 41 frame: 3 -> 1
Num 44 frame: 3 -> 1
Num 48 frame: 3 -> 1
Num 53 frame: 3 -> 1
Num 55 frame: 3 -> 1
Num 62 frame: 2 -> 4
113.43574717560527
Num 67 frame: 3 -> 1
Num 77 frame: 3 -> 1
Num 80 frame: 3 -> 1
Num 81 frame: 3 -> 1
Num 86 frame: 3 -> 1
Num 88 frame: 3 -> 1
Num 92 frame: 1 -> 3
Num 94 frame: 1 -> 3
Num 99 frame: 1 -> 3
Num 109 frame: 1 -> 3
173.72487622968632
Num 119 frame: 1 -> 3
Num 126 frame: 1 -> 3
157.67982288345567
Num 132 frame: 3 -> 1
Num 139 frame: 3 -> 1
Num 146 frame: 3 -> 1
Num 147 frame: 3 -> 1
Num 150 frame: 3 -> 1
Num 160 frame: 1 -> 3
170.79359566205716
Num 184 frame: 1 -> 3
Num 185 frame: 1 -> 3
Num 187 frame: 1 -> 3
Num 188 frame: 1 -> 3
Num 190 frame: 1 -> 3
Num 196 frame: 3 -> 1
150.50470227113016
Num 223 frame: 3 -> 1
172.8399048557522
Num 243 frame: 1 -> 3
Num 265 frame: 3 -> 1
Num 272 frame: 3 -> 1
169.8608278

In [56]:
# Select the index of the points (in the simplified trajectory) with the greatest theta
# Large theta is associated with greatest change in direction.
idx_simple_trajectory = np.where(theta>min_angle)[0]+1

idx_filtered = []
filtered_events = []
ie = {}

for index in idx_simple_trajectory:
    event_type = is_bounce[index - 1]  # "shot"/"bounce"/False等
    if event_type in ("shot", "bounce"):
        idx_filtered.append(index)
        filtered_events.append(event_type)    #这是在simplified的bounce

# Return real indices of bouncing points
ix = find_indices(points, list(zip(sx, sy)), idx_filtered)
for ixx in range(len(ix)):
    ie[ix[ixx]] = filtered_events[ixx]

In [57]:
ix # [8, 17, 64, 74, 114, 145, 161, 188, 189, 190, 191, 215, 224, 250, 261, 280, 292, 313, 328]
   # [8, 17, 64, 74, 114, 145, 161, 188, 189, 190, 191, 215, 224, 250, 261, 280, 292, 313, 328]

[9,
 17,
 63,
 64,
 102,
 114,
 115,
 145,
 161,
 178,
 188,
 189,
 190,
 191,
 215,
 224,
 230,
 250,
 261,
 280,
 292,
 313,
 328,
 343,
 353]